In [13]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
import io
import sys 
import subprocess

In [14]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
ds_api_key = os.getenv('DEEPSEEK_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')


In [15]:
MODEL_MAP = {
    "GPT": {
        "model": "gpt-4o-mini",
        "key": openai_api_key,
        "endpoint": "https://api.openai.com/v1",
    },
    "CLAUDE_3_5_SONNET": {
        "model": "claude-3-5-sonnet-20240620",
        "key": anthropic_api_key,
        "endpoint": "https://api.anthropic.com/v1"
    },
    "Grok": {
        "model": "grok-beta",
        "key": grok_api_key,
        "endpoint": "https://api.grok.com/v1"
    },   
   "DeepSeek": {
        "model": "deepseek-coder",
        "key": ds_api_key,
        "endpoint": "https://api.deepseek.com/v1",
    },
    "Google": {
        "model": "gemini-2.0-flash-exp",
        "key": google_api_key,
        "endpoint": "https://generativelanguage.googleapis.com/v1beta/openai"
    },
}

In [ ]:
class PortCode:
    def __init__(self, progress=None, model_name=MODEL_MAP["GPT"]):
        self.progress = progress
        self.model_deets = model_name
        self.model = OpenAI(
            api_key=model_name["key"],
            base_url=model_name["endpoint"]
        )
        self.cpp_code = ""
        
    def update_progress(self, value, desc=""):
        if self.progress:
            self.progress(value, desc=desc)
        
    def port_python_to_cpp(self, python_code):
        self.update_progress(0.3, desc="Converting Python to C++...")
        
        system_prompt = """
        Your task is to convert Python code into high performance C++ code.
        Respond only with C++ code. Do not provide any explanation other than occasional comments.
        The C++ response needs to produce an identical output in the fastest possible time.
        """
        
        user_prompt = f"""
        Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
        Respond only with C++ code.
        Python code to port:

        ```python
        {python_code}
        ```
        """
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        
        try:
            response = self.model.chat.completions.create(
                model=self.model_deets["model"],
                messages=messages
            )
            
            cpp_code = response.choices[0].message.content
            cpp_code = cpp_code.replace('```cpp', '').replace('```', '').strip()
            
            self.cpp_code = cpp_code
            
            self.update_progress(1.0, desc="Conversion complete!")
            return cpp_code
            
        except Exception as e:
            error_msg = f"Error converting code: {str(e)}"
            self.update_progress(1.0, desc="Conversion failed!")
            return error_msg
    
    def run_python_code(self, python_code):
        self.update_progress(0.1, desc="Running Python code...")
        
        globals_dict = {"__builtins__": __builtins__}
        buffer = io.StringIO()
        old_stdout = sys.stdout
        sys.stdout = buffer
        
        try:
            exec(python_code, globals_dict)
            output = buffer.getvalue()
            self.update_progress(1.0, desc="Python execution complete!")
        except Exception as e:
            output = f"Error: {e}"
            self.update_progress(1.0, desc="Python execution failed!")
        finally:
            sys.stdout = old_stdout
        
        return output
    
    def compile_cpp(self, cpp_code=None):
        if cpp_code is None:
            cpp_code = self.cpp_code
            
        if not cpp_code:
            return "No C++ code to compile. Please convert Python code first."
            
        self.update_progress(0.5, desc="Compiling C++ code...")
        
        with open("main.cpp", "w") as f:
            f.write(cpp_code)
        
        compile_command = [
            "clang++", "-std=c++17", "-Ofast", "-mcpu=native", 
            "-flto=thin", "-fvisibility=hidden", "-DNDEBUG", 
            "main.cpp", "-o", "main"
        ]
        
        try:
            subprocess.run(compile_command, check=True, text=True, capture_output=True)
            self.update_progress(1.0, desc="C++ compilation complete!")
            return "Compilation successful!"
            
        except subprocess.CalledProcessError as e:
            error_msg = f"Compilation error: {e.stderr}"
            self.update_progress(1.0, desc="C++ compilation failed!")
            return error_msg
        except Exception as e:
            error_msg = f"Error: {str(e)}"
            self.update_progress(1.0, desc="C++ compilation failed!")
            return error_msg
    
    def run_cpp(self):
        self.update_progress(0.1, desc="Running C++ code...")
        
        run_command = ["./main"]
        
        try:
            if not os.path.exists("./main"):
                return "No compiled executable found. Please compile C++ code first."
            
            run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
            print("hello .....")
            self.update_progress(1.0, desc="C++ execution complete!")
            return run_result.stdout
            
        except subprocess.CalledProcessError as e:
            error_msg = f"Runtime error: {e.stderr}"
            self.update_progress(1.0, desc="C++ execution failed!")
            return error_msg
        except Exception as e:
            error_msg = f"Error: {str(e)}"
            self.update_progress(1.0, desc="C++ execution failed!")
            return error_msg
    
    def compile_and_run_cpp(self, cpp_code=None):
        """Compile and run C++ code in one step"""
        if cpp_code is None:
            cpp_code = self.cpp_code
            
        if not cpp_code:
            return "No C++ code to compile and run. Please convert Python code first."
            
        compile_result = self.compile_cpp(cpp_code)
        if "error" in compile_result.lower():
            return compile_result
        
        return self.run_cpp()
    
    def get_cpp_code(self):
        """Get the stored C++ code"""
        return self.cpp_code
    
    def set_cpp_code(self, cpp_code):
        """Manually set C++ code"""
        self.cpp_code = cpp_code

In [37]:

class Interface:
    def __init__(self):
        self.port_code = PortCode(gr.Progress())
        
    def create_interface(self):
        with gr.Blocks(title="Code Porter") as interface:
            gr.Markdown("# 🚀 Python to C++ Converter")
            
            with gr.Row():
                python_input = gr.TextArea(label="Python Code", lines=15)
                cpp_output = gr.TextArea(label="C++ Code", lines=15, interactive=False)
            
            with gr.Row():
                python_result = gr.TextArea(label="Python Output", lines=4, interactive=False)
                cpp_result = gr.TextArea(label="C++ Output", lines=4, interactive=False)
            
            with gr.Row():
                run_python_btn = gr.Button("Run Python")
                run_cpp_btn = gr.Button("Run C++")
            
            with gr.Row():
                model_dropdown = gr.Dropdown(MODEL_MAP.keys(), value="GPT", label="Model")
            
            with gr.Row():
                convert_btn = gr.Button("Convert", variant="primary")
            
            # Events
            convert_btn.click(self.convert_code, [python_input, model_dropdown], cpp_output)
            run_python_btn.click(self.run_python, python_input, python_result)
            run_cpp_btn.click(self.run_cpp, cpp_output, cpp_result)
            model_dropdown.change(self.update_model, model_dropdown, None)
        
        return interface
    
    def convert_code(self, python_code, model_name):
        self.port_code = PortCode(model_name=MODEL_MAP[model_name])
        return self.port_code.port_python_to_cpp(python_code)
    
    def run_python(self, python_code):
        return self.port_code.run_python_code(python_code)
    
    def run_cpp(self, cpp_code):
        self.port_code.set_cpp_code(cpp_code)
        return self.port_code.compile_and_run_cpp()
    
    def update_model(self, model_name):
        self.port_code = PortCode(model_name=MODEL_MAP[model_name])
    
    def launch(self, inbrowser=False):
        self.create_interface().launch(inbrowser=inbrowser)

In [38]:
I = Interface()
I.launch()

* Running on local URL:  http://127.0.0.1:7906
* To create a public link, set `share=True` in `launch()`.
